In [ ]:
import pandas as pd
import yake
import re

In [ ]:
# Importing dataset, enter the path of the csv file
sum_df = pd.read_csv("dataset/train.csv")
sum_df['NUMERIC_DATA'] = ''
sum_df=sum_df.dropna()

In [ ]:
def getKeywords(text):

    # Create a YAKE keyword extractor
    kw_extractor = yake.KeywordExtractor(lan="en", n=1, dedupLim=0.9, top=20)

    # Extract numeric values and replace with "NUMERIC"
    numeric_values = []
    text_tokens = []
    for token in text.split():
        if token.replace(".", "").isnumeric():
            numeric_values.append(token)
            text_tokens.append("NUMERIC")
        else:
            text_tokens.append(token)

    # Convert text tokens back to a string
    text = " ".join(text_tokens)

    # Extract keywords from the modified text
    keywords = kw_extractor.extract_keywords(text)

    # Replace "NUMERIC" with the original numeric values
    for i, keyword in enumerate(keywords):
        if keyword[0] == "NUMERIC":
            keywords[i] = (numeric_values.pop(0), keyword[1])

    opstr=""
    for i in keywords:
        opstr=opstr+i[0]+","
    
    # return the keywords
    return opstr

In [ ]:
def numeric_data_extractor(description):
    # description = "Made,Hard,Polycarbonate,Plastic,3,Case,cover,sides,back,mobile,printed,bumps,protects,scratches,premier,"

    length_pattern = r"\b(?:\d+(?:\s\d+/\d+)?(?:\.\d+)?|\d+/\d+)(?:\s*\*\s*(?:\d+(?:\s\d+/\d+)?(?:\.\d+)?|\d+/\d+))*\s*(?:cm|mm|m|in(?:ch(?:es)?)?|ft|foot|feet|yd|yard|')?\b"
    # length_pattern = r"\b(?:\d+(?:\.\d+)?|\d+/\d+)(?:\s*\*\s*(?:\d+(?:\.\d+)?|\d+/\d+))*\s*(?:cm|mm|m|in(?:ch(?:es)?)?|ft|foot|feet|yd|yard|')?\b"
    # length_pattern = r'(\d+(?:\.\d+)?(?:/\d+)?(?:\s*\*\s*\d+(?:\.\d+)?(?:/\d+)?){0,2})(?:\s*(?:feet|ft|meters|m|centimeters|cm|inches|in|"))?'

    lengths = re.findall(length_pattern, description, re.IGNORECASE)

    

    opstr=""
    for i in lengths:
        opstr=opstr+i+","
    
    return opstr

In [ ]:
# Set the column name
column_name_1 = "TITLE"
column_name_2 = "BULLET_POINTS"
column_name_3 = "DESCRIPTION"

# Loop through each row in the dataframe
for index, row in sum_df.iterrows():
    # Extract the text to be summarized from the column
    text_1 = row[column_name_1]
    text_2 = row[column_name_2]
    text_3 = row[column_name_3]

    inpstr= text_1+" "+text_2+" "+text_3
    
    # Use get numeric data
    ndata=numeric_data_extractor(inpstr)

    # Replace the original text with the summary in the same column
    sum_df.at[index, 'NUMERIC_DATA'] = ndata

In [ ]:
# Set the column name
column_name = "DESCRIPTION"

# Loop through each row in the dataframe
for index, row in sum_df.iterrows():
    # Extract the text to be summarized from the column
    text = row[column_name]
    
    # Use yake to extract 20 keywords 
    kw=getKeywords(text)

    # Replace the original text with the summary in the same column
    sum_df.at[index, column_name] = kw

In [ ]:
# Set the column name
column_name = "BULLET_POINTS"

# Loop through each row in the dataframe
for index, row in sum_df.iterrows():
    # Extract the text to be summarized from the column
    text = row[column_name]
    
    # Use yake to extract 20 keywords 
    kw=getKeywords(text)

    # Replace the original text with the summary in the same column
    sum_df.at[index, column_name] = kw